In [1]:
import jax
import jax.numpy as jnp

In [15]:
import jax.numpy as jnp


def matmul_true_int8(lhs, rhs):
  assert lhs.dtype == jnp.int8
  print(lhs)
  print(rhs)
  assert rhs.dtype == jnp.int8
  result = jnp.matmul(lhs, rhs, preferred_element_type=jnp.int32)
  assert result.dtype == jnp.int32
  print(result)
  return result

In [16]:
def gen_matrix(rows, columns):
  import numpy as np
  return np.random.normal(size=(rows, columns)).reshape((rows, columns))


batch_size = 3
channels_in = 4
channels_out = 5


a = gen_matrix(batch_size, channels_in) # Activations
w = gen_matrix(channels_in, channels_out) # Weights

In [23]:
def aqt_matmul_int8(a, w):
  max_int8 = 127
  
  def quant_int8(x):
    return jnp.clip(jnp.round(x), -max_int8, max_int8).astype(jnp.int8)


  a_s = max_int8 / jnp.max(jnp.abs(a), axis=1, keepdims=True)
  w_s = max_int8 / jnp.max(jnp.abs(w), axis=0, keepdims=True)

  print(a_s.dtype)
  # int8 matmul with int32 accumulator
  result = matmul_true_int8(quant_int8(a * a_s), quant_int8(w * w_s)) / (a_s * w_s)
  assert result.shape == (batch_size, channels_out)

  return result

In [24]:
# Test
print(f"jnp.matmul(a, w):\n", jnp.matmul(a, w))
print(f"aqt_matmul_int8(a, w):\n", aqt_matmul_int8(a, w))

jnp.matmul(a, w):
 [[ 0.405507    2.8269942   1.37993    -0.33883294  1.271378  ]
 [-4.054617    0.2763729  -0.33638394  2.1558146   0.8080269 ]
 [ 0.5041073  -2.035844    3.8957129  -2.39001    -1.8010538 ]]
float32
[[ -53   18    5  127]
 [  -8 -116 -127   32]
 [-127   92  -45  -54]]
[[ -24   88 -119   55   81]
 [  97   14  127 -127 -102]
 [ 127    5 -102   40   67]
 [  -8  127  -18   27  127]]
[[  2637  11742   5797  -1572  10335]
 [-27445   1101  -1402  10076   6739]
 [  6689 -16971  32359 -21927 -29544]]
aqt_matmul_int8(a, w):
 [[ 0.39420462  2.8258045   1.398726   -0.34337255  1.2642894 ]
 [-4.05275     0.26173535 -0.33415842  2.1740837   0.81434137]
 [ 0.49942753 -2.0398922   3.8996367  -2.3921695  -1.8051156 ]]
